In [5]:
import keras
import dataset
import cv2 as cv

In [2]:
IMAGE_SIZE = (152, 152)
CHANNELS = 3
NUM_CLASSES = 8631

BATCH_SIZE = 1024
LEARN_RATE = 0.01 * (BATCH_SIZE / 128)
MOMENTUM = 0.9
EPOCHS = 15

In [3]:
def create_deepface():
    wt_init = keras.initializers.RandomNormal(mean=0, stddev=0.01)
    bias_init = keras.initializers.Constant(value=0.5)

    def conv2d_layer(**args):
        return keras.layers.Conv2D(**args,
                                   kernel_initializer=wt_init,
                                   bias_initializer=bias_init,
                                   activation=keras.activations.relu)
    def lc2d_layer(**args):
        return keras.layers.LocallyConnected2D(**args,
                                               kernel_initializer=wt_init,
                                               bias_initializer=bias_init,
                                               activation=keras.activations.relu)
    def dense_layer(**args):
        return keras.layers.Dense(**args,
                                  kernel_initializer=wt_init,
                                  bias_initializer=bias_init)


    deepface = keras.models.Sequential([
        keras.layers.InputLayer(input_shape=(*IMAGE_SIZE, CHANNELS), name='I0'),
        conv2d_layer(filters=32, kernel_size=11, name='C1'),
        keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same',  name='M2'),
        conv2d_layer(filters=16, kernel_size=9, name='C3'),
        lc2d_layer(filters=16, kernel_size=9, name='L4'),
        lc2d_layer(filters=16, kernel_size=7, strides=2, name='L5'),
        lc2d_layer(filters=16, kernel_size=5, name='L6'),
        keras.layers.Flatten(name='F0'),
        dense_layer(units=4096, activation=keras.activations.relu, name='F7'),
        keras.layers.Dropout(rate=0.4, name='D0'),
        dense_layer(units=NUM_CLASSES, activation=keras.activations.softmax, name='F8')
        ], name='DeepFace')

    sgd_opt = keras.optimizers.SGD(lr=LEARN_RATE, momentum=MOMENTUM)
    cce_loss = keras.losses.categorical_crossentropy

    deepface.compile(optimizer=sgd_opt, loss=cce_loss, metrics=['accuracy'])
    return deepface

In [3]:
labels, trainX, trainy, valX, valy = dataset.read_images('data/dataset', IMAGE_SIZE, 50)

In [4]:
print(trainX.shape)
print(trainy.shape)
print(valX.shape)
print(valy.shape)

(160, 152, 152, 3)
(160,)
(40, 152, 152, 3)
(40,)


In [12]:
i=12
cv.imshow('test', trainX[i])
print(trainy[i])
cv.waitKey(0)
cv.destroyAllWindows()

3
